In [92]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [93]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [94]:
import re 
text = "Hello, world. This, is a test."
result = re.split(r'([,.]|\s)', text)


result = [item for item in result if item.strip()]
result

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']

In [95]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
preprocessed[:30]

4690


['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius',
 '--',
 'though',
 'a',
 'good',
 'fellow',
 'enough',
 '--',
 'so',
 'it',
 'was',
 'no',
 'great',
 'surprise',
 'to',
 'me',
 'to',
 'hear',
 'that',
 ',',
 'in']

## 2.3 Converting tokens into token IDs

In [96]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
vocab_size

1130

In [97]:
vocab = { token:integer for integer , token in enumerate(all_words) }
len(vocab)

1130

In [98]:
class SimpleTokenizerV1:
  def __init__(self,  vocab):
    self.str_to_int = vocab 
    self.int_to_str = {i:s for s,i in vocab.items()}
  
  def encode(self , text ):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)                                
    preprocessed = [
        item.strip() for item in preprocessed if item.strip()
    ]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids
  def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text
  

In [99]:
tokenizer = SimpleTokenizerV1(vocab)
text = """ "It's the last he painted , you know , " 
Mr.s Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
ids

[1,
 56,
 2,
 850,
 988,
 602,
 533,
 746,
 5,
 1126,
 596,
 5,
 1,
 66,
 7,
 850,
 38,
 851,
 1108,
 754,
 793,
 7]

In [100]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mr. s Gisburn said with pardonable pride.'

In [101]:
text = "Hello , do you like tea?"
# tokenizer.encode((text))

In [102]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])
vocab = { token:integer for integer,token in enumerate(all_tokens) }
len(vocab.items())

1132

In [103]:
list(vocab.items())[-5:]


[('younger', 1127),
 ('your', 1128),
 ('yourself', 1129),
 ('<|endoftext|>', 1130),
 ('<|unk|>', 1131)]

In [104]:
class SimpleTokenizerV2:
  def __init__(self, vocab):
    self.str_to_int = vocab 
    self.int_to_str = { i:s for s,i in vocab.items()}
  def encode(self, text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    preprocessed = [item if item in self.str_to_int  else "<|unk|>" for item in preprocessed]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids 
  def decode(self, ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
    return text

In [105]:
tokenizer = SimpleTokenizerV2(vocab)
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
text

'Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.'

In [106]:
tokenizer.encode(text)
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

In [107]:
import importlib 
import tiktoken 
importlib.metadata.version("tiktoken")

'0.8.0'

In [108]:
tokenizer = tiktoken.get_encoding("gpt2")

In [109]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [110]:
strings = tokenizer.decode(integers)
strings

'Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.'

## 2.6 Data sampling with a sliding window

In [111]:
with open("the-verdict.txt" , "r" , encoding="utf-8") as f:
  raw_text = f.read()
enc_text = tokenizer.encode(raw_text)
len(enc_text)

5145

In [112]:
enc_sample = enc_text[50:]

context_size = 4        
x = enc_sample[:context_size]
y = enc_sample[1:context_size + 1] 
print(f"x: { x } ")
print(f"y:     { y }")

x: [290, 4920, 2241, 287] 
y:     [4920, 2241, 287, 257]


In [113]:
for i in range(1 , context_size + 1):
  context = enc_sample[:i]
  desired = enc_sample[i]  
  print(context , "---->" , desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [114]:
for i in range(1 , context_size + 1):
  context = enc_sample[:i]
  desired = enc_sample[i]  
  print(tokenizer.decode(context) , "---->" , tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [115]:
import torch
from torch.utils.data import Dataset , DataLoader

class GPTDatasetV1(Dataset):
  def __init__(self , txt , tokenizer , max_length , stride):
    self.input_ids = [] 
    self.target_ids = [] 
    token_ids = tokenizer.encode(txt , allowed_special={"<|endoftext|>"})
    for i  in range(0 , len(token_ids) - max_length , stride):
      input_chunk = token_ids[i:i + max_length]
      target_chunk = token_ids[i +  1 : i + max_length + 1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))
    
  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self, idx):
    return self.input_ids[idx] , self.target_ids[idx]
    

In [116]:
def create_dataloader_v1(txt , batch_size , max_length=256 , stride = 128 , shuffle = True , drop_last=True , num_workers = 0):
  tokenizer = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV1(txt , tokenizer , max_length , stride)
  dataloader = DataLoader(dataset , batch_size = batch_size , shuffle=shuffle , drop_last=drop_last , num_workers=num_workers)
  return dataloader

In [117]:
with open("./the-verdict.txt" , "r" , encoding="utf-8") as f:
  raw_text = f.read() 
data_loader  = create_dataloader_v1(raw_text , batch_size= 1 , max_length=4 , stride=1 , shuffle=False)
data_iter = iter(data_loader)
first_batch = next(data_iter)
first_batch

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]

In [118]:
dataloader = create_dataloader_v1(
 raw_text, batch_size=8, max_length=4, stride=4,
 shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:\n tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 Creating token embeddings

In [119]:
input_ids = torch.tensor([2,3,5,1])
vocab_size = 6 
output_dim = 3 
torch.manual_seed(123)
embeding_layer = torch.nn.Embedding(vocab_size , output_dim)

In [120]:
embeding_layer.weight

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)

In [121]:
print(embeding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


## 2.8 Encoding word positions

In [122]:
vocab_size = 50257 
output_dim = 256 
token_embedding_layer = torch.nn.Embedding(vocab_size , output_dim)

In [123]:
max_length = 4 
dataloader = create_dataloader_v1(raw_text ,batch_size=8 , max_length=max_length , stride = max_length , shuffle = False)
data_iter = iter(dataloader)
inputs , targets = next(data_iter)


In [124]:
print("Token IDs:\n" , inputs) 
print("\nInputs shape" , inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape torch.Size([8, 4])


In [126]:
token_embeddings = token_embedding_layer(inputs)
token_embeddings.shape

torch.Size([8, 4, 256])

In [127]:
context_length = max_length 
pos_embedding_layer = torch.nn.Embedding(context_length , output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
pos_embeddings.shape

torch.Size([4, 256])

In [128]:
input_embeddings = token_embeddings + pos_embeddings
input_embeddings.shape

torch.Size([8, 4, 256])